# Investiogation of Synthetic Images

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import numpy as np
import re


## 1- Initial set of images - 6/2/2017


In [2]:
def text2coor(textpath):
    f=open(textpath,'r')
    log=f.read()
    coor=np.asarray(re.split('\(|, |\)|\n',log))
    coor=np.delete(coor,np.where(coor=='')[0])
    coor=np.reshape(coor,(-1,3)).astype(np.float)
    return coor

def normalize(inarray,x,y,m=0,M=0):
    if(m==0 and M==0):
        m=np.min(inarray)
        M=np.max(inarray)
    print('Normalization started with m=%.2f and M=%.2f'%(m,M))
    A=np.array([[m,1],[M,1]])
    b=np.array([[x],[y]])
    b2=np.linalg.solve(A,b)
    return inarray*b2[0]+b2[1];


[[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]

[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]

In [3]:
print([0,1,4]==[0,1,4])

True


In [4]:
def prune_data(coor,corners):
    if((corners==[0,0,0,0,0,0]).all):
        return coor
    
    [x1,x2,y1,y2,z1,z2]=corners
    if x1>x2:
        temp=x1
        x1=x2
        x2=temp
    if y1>y2:
        temp=y1
        y1=y2
        y2=tmp
    if z1>z2:
        temp=z1
        z1=z2
        z2=temp
        
    idx=np.zeros(coor.shape[0])
    count=0
    for k in range(len(idx)):
        [x,y,z]=coor[k,:]
        if(x<x1 or x>x2 or y<y1 or y>y2 or z<z1 or z>z2):
            idx[count]=k
            count=count+1
    idx=idx[:count]
    coor=np.delete(coor,idx,axis=0)
    
    return coor

In [5]:
def featExtract(rootdir,logdir,rateddir,subdirslog,subdirsrated,nrmalize=True,\
                m=[[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]],):
    label=np.zeros((2000,1))
    room=np.zeros((2000,1))
    fvec = np.zeros((2000,4000,4))
    imcount=np.zeros((2000,1))

    for k in range(len(rateddir)):
        for l in range(len(subdirsrated)):
            for cir in range(1,10):
                fnames=os.listdir("".join((rootdir,rateddir[k],subdirsrated[l],'\\',str(cir))))
                for fname in fnames:
                    if fname.endswith('.png'):
                        imNumber=int(fname.split('_')[0])+k*1000
                        logpath="".join((rootdir,logdir[k],subdirslog[l],'\\',str(imNumber-k*1000),'.txt'))
                        coor = text2coor(logpath);
                        coor=prune_data(coor,m[l])
                        fvec[imNumber,:coor.shape[0],0]=1
                        fvec[imNumber,:coor.shape[0],1:4]=coor
                        del coor
                        imcount[imNumber] += 1
                        label[imNumber] += cir
                        room[imNumber]=l+1;



    label=np.delete(label,np.where(imcount==0)[0])
    room=np.delete(room,np.where(imcount==0)[0])
    fvec=np.delete(fvec,np.where(imcount==0)[0],axis=0)

    imcount=np.delete(imcount,np.where(imcount==0)[0])

    print(''.join(('Number of images is ',str(sum(imcount)))))
    print(''.join(('Number of frames is ',str(len(imcount)))))
    label=label/imcount
    print('Mean CIR values for scenes')
    print(label)

    print('Rooms of scenes')
    print(room)

    print('Shape of the feature vector')
    print(fvec.shape)

    #print('Feature vector for the 58th image')
    #print(fvec[58,:,:])

    fvec_n=fvec.copy()
    
    if(nrmalize):
        fvec_n[room==1,:,1]=normalize(fvec[room==1,:,1],-1,1,m=m[0][0],M=m[0][1])
        fvec_n[room==1,:,2]=normalize(fvec[room==1,:,2],-1,1,m=m[0][2],M=m[0][3])
        fvec_n[room==1,:,3]=normalize(fvec[room==1,:,3],-1,1,m=m[0][4],M=m[0][5])

        if(len(subdirslog)>1):
            fvec_n[room==2,:,1]=normalize(fvec[room==2,:,1],-1,1,m=m[1][0],M=m[1][1])
            fvec_n[room==2,:,2]=normalize(fvec[room==2,:,2],-1,1,m=m[1][2],M=m[1][3])
            fvec_n[room==2,:,3]=normalize(fvec[room==2,:,3],-1,1,m=m[1][4],M=m[1][5])

        if(len(subdirslog)>2):
            fvec_n[room==3,:,1]=normalize(fvec[room==3,:,1],-1,1,m=m[2][0],M=m[2][1])
            fvec_n[room==3,:,2]=normalize(fvec[room==3,:,2],-1,1,m=m[2][2],M=m[2][3])
            fvec_n[room==3,:,3]=normalize(fvec[room==3,:,3],-1,1,m=m[2][4],M=m[2][5])
        
        

    #print('Normalized feature vector for the 58th image')
    #print(fvec_n[58,:,:])
    return fvec_n,label,room

rootdir = 'Z:\Grants\Image Project\Data_Sets\Synthetic'
logdir = ['\Old\Rural Home 5.4.17','\Old\RuralHome4.26']
rateddir=['\Rated\Ruralhome5.4RATED','\Rated\Ruralrated4.26']
subdirslog=['\Room1','\Room2','\Room3']
subdirsrated=['\Room 1','\Room 2','\Room 3']

#[fvec1_n,label1,room1]=featExtract(rootdir,logdir,rateddir,subdirslog,subdirsrated)

In [6]:
def heatmap(x,y,z,plot_b=False):

    clutter_img=np.zeros((555,555))

    for k in range(len(x)):
        clutter_img[x[k]-10:x[k]+10,z[k]-10:z[k]+10]= \
            np.maximum(clutter_img[x[k]-10:x[k]+10,z[k]-10:z[k]+10],y[k])
    if(plot_b==True):
        imgplot=plt.imshow(clutter_img)
        plt.colorbar(imgplot)
    return clutter_img

Let us define better feature vectors

In [7]:
def feat_transform(fvec):
    color_levels=np.array([0,.3,.6,.7,.8,.9,.95,1])
    numScenes = fvec.shape[0]
    fvec_stat=np.zeros((numScenes,14))
    fvec_c = np.zeros(14)
    for k in range(numScenes):
        coors=fvec[k,:,:]
        coors=coors[coors[:,0]==1,:]
        coors=coors[:,1:4]

        x=coors[:,0]
        y=coors[:,1]
        z=coors[:,2]

        x=(11+np.round(256*(x+1))).astype(np.int)
        z=(11+np.round(256*(z+1))).astype(np.int)
        y=(y+1)/2.

        top_height=np.zeros(6)
        if(len(y)>0):
            top_height[0]=y[0]
            top_height[1]=np.mean(y[:10])
            top_height[2]=np.mean(y[:20])
            top_height[3]=np.mean(y[:50])
            top_height[4]=np.mean(y[:100])
            top_height[5]=np.mean(y[:1000])


        clutter_img=heatmap(x,y,z)
        clutter_levels=np.histogram(clutter_img,color_levels)[0]

        numObj=len(y)

        fvec_c[0]=numObj/1500
        fvec_c[1:8]=clutter_levels/(555*555/10)
        fvec_c[8:]=top_height

        fvec_stat[k,:]=fvec_c

    fvec_stat_01=fvec_stat/(np.ones((fvec_stat.shape[0],1))*np.max(fvec_stat,axis=0))
    return fvec_stat
    
#fvec1_stat=feat_transform(fvec1_n)
#print(fvec1_stat)
    




Now, let us try some machine learning

In [8]:
from sklearn import linear_model

def cross_val(clf,X,y,cv=4):
    n=len(y)
    idx=np.random.permutation(n)
    X=X[idx,:]
    y=y[idx]
    cv_id=np.round(np.linspace(0,n,cv+1))

    y_pred=np.zeros(y.shape)
    for k in range(len(cv_id)-1):
        id1=int(cv_id[k])
        id2=int(cv_id[k+1])
        X_test=X[id1:id2,:]
        X_tr=X.copy()
        X_tr=np.delete(X_tr,range(id1,id2),axis=0)
        y_tr=y.copy()
        y_tr=np.delete(y_tr,range(id1,id2))

        clf.fit(X_tr,y_tr)
        y_pred[id1:id2] = clf.predict(X_test)
    return y,y_pred
       
def cross_val2(clf,X,y,cv=4):
    n=len(y)
    idx=np.random.permutation(n)
    X=X[idx,:]
    y=y[idx]
    cv_id=np.round(np.linspace(0,n,cv+1))
    
    cir1=np.zeros(cv)
    cir2=np.zeros(cv)
    for k in range(len(cv_id)-1):
        id1=int(cv_id[k])
        id2=int(cv_id[k+1])
        X_tr=X[id1:id2,:]
        X_test=X.copy()
        X_test=np.delete(X_test,range(id1,id2),axis=0)
        y_tr=y[id1:id2]
        y_test=y.copy()
        y_test=np.delete(y_test,range(id1,id2))

        clf.fit(X_tr,y_tr)
        y_pred = clf.predict(X_test)
        cir1[k]=np.mean((np.abs(np.round(y_pred)-y_test)<=1).astype(np.float))
        cir2[k]=np.mean((np.abs(np.round(y_pred)-y_test)<=2).astype(np.float))
    return cir1,cir2
        
'''
cv=6
regr = linear_model.LinearRegression()
[y_cv, y_pred]=cross_val(regr,fvec1_stat,label1,cv=cv) 
cir1= np.mean((np.abs(np.round(y_pred)-y_cv)<=1).astype(np.float))
cir2= np.mean((np.abs(np.round(y_pred)-y_cv)<=2).astype(np.float))
print('%d-fold CV: CIR-1: %.2f, CIR-2: %.3f'%(cv,cir1,cir2))
[cir1,cir2]=cross_val2(regr,fvec1_stat,label1,cv=cv)
print('Using %d samples:'%(int(np.round(len(label1)/cv))))
print("CIR-1: ",cir1)
print("CIR-2: ",cir2)
'''

'\ncv=6\nregr = linear_model.LinearRegression()\n[y_cv, y_pred]=cross_val(regr,fvec1_stat,label1,cv=cv) \ncir1= np.mean((np.abs(np.round(y_pred)-y_cv)<=1).astype(np.float))\ncir2= np.mean((np.abs(np.round(y_pred)-y_cv)<=2).astype(np.float))\nprint(\'%d-fold CV: CIR-1: %.2f, CIR-2: %.3f\'%(cv,cir1,cir2))\n[cir1,cir2]=cross_val2(regr,fvec1_stat,label1,cv=cv)\nprint(\'Using %d samples:\'%(int(np.round(len(label1)/cv))))\nprint("CIR-1: ",cir1)\nprint("CIR-2: ",cir2)\n'

In [9]:
#Some trials
logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\to-be-Rated\\RuralHometo-be-rated_08-Jun-2017\\logs\\fr2_log.txt'
[coor,scle,sze,rot,corners1] = text2feat(logpath)

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\to-be-Rated\\RuralHometo-be-rated_08-Jun-2017\\logs\\fr53_log.txt'
[coor,scle,sze,rot,corners2] = text2feat(logpath)

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\to-be-Rated\\RuralHometo-be-rated_08-Jun-2017\\logs\\fr100_log.txt'
[coor,scle,sze,rot,corners3] = text2feat(logpath)

corners1=[corners1]
corners2=[corners2]
corners3=[corners3]
print(corners1)
print(corners2)
print(corners3)

rootdir = 'Z:\Grants\Image Project\Data_Sets\Synthetic'
logdir = ['\Old\Rural Home 5.4.17']
rateddir=['\Rated\Ruralhome5.4RATED']
subdirslog=['\Room1']
subdirsrated=['\Room 1']

[fvec1_n,label1,room1]=featExtract(rootdir,logdir,rateddir,subdirslog,subdirsrated,nrmalize=False,m=corners1)
fvec1_stat=feat_transform(fvec1_n)

rootdir = 'Z:\Grants\Image Project\Data_Sets\Synthetic'
logdir = ['\Old\Rural Home 5.4.17']
rateddir=['\Rated\Ruralhome5.4RATED']
subdirslog=['\Room2']
subdirsrated=['\Room 2']

[fvec2_n,label2,room2]=featExtract(rootdir,logdir,rateddir,subdirslog,subdirsrated,nrmalize=False,m=corners2)
fvec2_stat=feat_transform(fvec2_n)

rootdir = 'Z:\Grants\Image Project\Data_Sets\Synthetic'
logdir = ['\Old\Rural Home 5.4.17']
rateddir=['\Rated\Ruralhome5.4RATED']
subdirslog=['\Room3']
subdirsrated=['\Room 3']

[fvec3_n,label3,room3]=featExtract(rootdir,logdir,rateddir,subdirslog,subdirsrated,nrmalize=False,m=corners3)
fvec3_stat=feat_transform(fvec3_n)


regr = linear_model.LinearRegression()
regr.fit(fvec1_stat,label1)
label2_pred=regr.predict(fvec2_stat)
label3_pred=regr.predict(fvec3_stat)
print('Predictions for Room2')
print(label2_pred)
print('Predictions for Room3')
print(label3_pred)

print('Results for Room2')
cir1= np.mean((np.abs(np.round(label2_pred)- label2)<=1).astype(np.float))
cir2= np.mean((np.abs(np.round(label2_pred)- label2)<=2).astype(np.float))
print('CIR-1: %.2f, CIR-2: %.3f'%(cir1,cir2))

print('Results for Room3')
cir1= np.mean((np.abs(np.round(label3_pred)- label3)<=1).astype(np.float))
cir2= np.mean((np.abs(np.round(label3_pred)- label3)<=2).astype(np.float))
print('CIR-1: %.2f, CIR-2: %.3f'%(cir1,cir2))


'''
cv=3
regr = linear_model.LinearRegression()
[y_cv, y_pred]=cross_val(regr,fvec1_stat,label1,cv=cv) 
cir1= np.mean((np.abs(np.round(y_pred)-y_cv)<=1).astype(np.float))
cir2= np.mean((np.abs(np.round(y_pred)-y_cv)<=2).astype(np.float))
print('%d-fold CV: CIR-1: %.2f, CIR-2: %.3f'%(cv,cir1,cir2))
[cir1,cir2]=cross_val2(regr,fvec1_stat,label1,cv=cv)
print('Using %d samples:'%(int(np.round(len(label)/cv))))
print("CIR-1: ",cir1)
print("CIR-2: ",cir2)
'''

NameError: name 'text2feat' is not defined

In [ ]:
m=[[1,2,3,4,5],[6,9,2]]
print(m[0][3],m[1][1])



## 2- Second Set - 6/13/17

In [9]:
#textpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\to-be-Rated\\RuralHometo-be-rated_08-Jun-2017\\logs\\fr22_log.txt'
def swap_elements(array,i,j):
    tmp=array[i]
    array[i]=array[j]
    array[j]=tmp
    return array

def text2feat(textpath):
    f=open(textpath,'r')
    log=f.read()
    feat=np.asarray(re.split('\(|, |\)|\n',log))
    feat=np.delete(feat,np.where(feat=='')[0])
    if(feat[2]=='Corner Cubes for Room Dimensions'):
        corners=np.asarray([feat[3],feat[6],feat[4],feat[10],feat[5],feat[8]]).astype(np.float)
    else:
        corners=np.asarray([feat[2],feat[5],feat[3],feat[9],feat[4],feat[7]]).astype(np.float)
    idx=np.char.equal(feat,'Rotation').astype(np.int)
    idx = np.argwhere(idx)[0][0]

    feat=feat[idx+1:]
    feat=np.reshape(feat,(-1,3)).astype(np.float)
    coor=feat[::4,:]
    scle=feat[1::4,:]
    sze=feat[2::4,:]
    rot=feat[3::4,:]

    if(corners[1]<corners[0]):
        corners=swap_elements(corners,0,1)
    if(corners[3]<corners[2]):
        corners=swap_elements(corners,2,3)
    if(corners[5]<corners[4]):
        corners=swap_elements(corners,4,5)
        
    return coor,scle,sze,rot,corners


In [10]:
#textpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\to-be-Rated\\RuralHometo-be-rated_08-Jun-2017\\logs\\fr22_log.txt'

def text2feat2(textpath):
    f=open(textpath,'r')
    log=f.read()
    feat=np.asarray(re.split('\(|, |\)|\n',log))
    feat=np.delete(feat,np.where(feat=='')[0])
    corners=np.asarray([feat[3],feat[6],feat[4],feat[10],feat[5],feat[8]]).astype(np.float)
    idx=np.char.equal(feat,'Rotation').astype(np.int)
    idx = np.argwhere(idx)[0][0]

    feat=feat[idx+1:]
    feat=np.reshape(feat,(-1,3)).astype(np.float)
    coor=feat[::4,:]
    scle=feat[1::4,:]
    sze=feat[2::4,:]
    rot=feat[3::4,:]

    return coor,scle,sze,rot,corners


In [11]:
def prune_data2(coor,scle,sze,rot,corners):
    [x1,x2,y1,y2,z1,z2]=corners
    if x1>x2:
        temp=x1
        x1=x2
        x2=temp
    if y1>y2:
        temp=y1
        y1=y2
        y2=temp
    if z1>z2:
        temp=z1
        z1=z2
        z2=temp
        
    idx=np.zeros(coor.shape[0])
    count=0
    for k in range(len(idx)):
        [x,y,z]=coor[k,:]
        if(x<x1 or x>x2 or y<y1 or y>y2 or z<z1 or z>z2):
            idx[count]=k
            count=count+1
    idx=idx[:count]
    coor=np.delete(coor,idx,axis=0)
    scle=np.delete(scle,idx,axis=0)
    sze=np.delete(sze,idx,axis=0)
    rot=np.delete(rot,idx,axis=0)
    
    return coor,scle,sze,rot

In [ ]:
a=np.random.rand(20,3)
print(np.prod(a,axis=1))
print(np.sum(np.prod(a,axis=1)))


In [12]:
     
def featExtract2(rootdir,subdirs,roomdirs,nrmalize=True,m=[[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]):
    label=np.zeros((50000,1))
    room=np.zeros((50000,1))
    fvec = np.zeros((50000,15000,4))
    fvolume=np.zeros((50000,1))
    imcount=np.zeros((50000,1))
    

    room_dims=[[m[0][1]-m[0][0],m[0][3]-m[0][2],m[0][5]-m[0][4]],
               [m[1][1]-m[1][0],m[1][3]-m[1][2],m[1][5]-m[1][4]],
               [m[2][1]-m[2][0],m[2][3]-m[2][2],m[2][5]-m[2][4]]]
    vol_room=[room_dims[0][0]*room_dims[0][1]*room_dims[0][2],
              room_dims[1][0]*room_dims[1][1]*room_dims[1][2],
              room_dims[2][0]*room_dims[2][1]*room_dims[2][2]]
    print(vol_room)
    
    for k in range(len(subdirs)):
            for l in range(len(roomdirs)):
                for cir in range(1,10):
                    fnames=os.listdir("".join((rootdir,subdirs[k],roomdirs[l],'\\',str(cir))))
                    for fname in fnames:
                        if fname.endswith('.png'):
                            imNumber=int(fname.split('.')[0][2:])+k*1000
                            logpath="".join((rootdir,subdirs[k],'\\logs\\fr',str(imNumber-k*1000),'_log.txt'))
                            [coor,scle,sze,rot,corners] = text2feat(logpath);
                            [coor,scle,sze,rot] = prune_data2(coor,scle,sze,rot,corners)
                            fvec[imNumber,:coor.shape[0],0]=1
                            fvec[imNumber,:coor.shape[0],1:4]=coor
                            sze_obj=scle*sze
                            fvolume[imNumber]=np.sum(np.prod(sze_obj,axis=1))
                            del coor
                            imcount[imNumber] += 1
                            label[imNumber] += cir
                            room[imNumber]=l+1;
                        



    label=np.delete(label,np.where(imcount==0)[0])
    room=np.delete(room,np.where(imcount==0)[0])
    fvec=np.delete(fvec,np.where(imcount==0)[0],axis=0)
    fvolume=np.delete(fvolume,np.where(imcount==0)[0],axis=0)

    imcount=np.delete(imcount,np.where(imcount==0)[0])
    
    fvolume[room==1]=fvolume[room==1]/vol_room[0]
    fvolume[room==2]=fvolume[room==2]/vol_room[1]
    fvolume[room==3]=fvolume[room==3]/vol_room[2]
    print(''.join(('Number of images is ',str(sum(imcount)))))
    print(''.join(('Number of frames is ',str(len(imcount)))))
    label=label/imcount
    print('Mean CIR values for scenes')
    print(label)

    print('Rooms of scenes')
    print(room)

    print('Shape of the feature vector')
    print(fvec.shape)

    #print('Feature vector for the 58th image')
    #print(fvec[58,:,:])

    fvec_n=fvec.copy()

    if(nrmalize):
            fvec_n[room==1,:,1]=normalize(fvec[room==1,:,1],-1,1,m=m[0][0],M=m[0][1])
            fvec_n[room==1,:,2]=normalize(fvec[room==1,:,2],-1,1,m=m[0][2],M=m[0][3])
            fvec_n[room==1,:,3]=normalize(fvec[room==1,:,3],-1,1,m=m[0][4],M=m[0][5])

            if(len(subdirslog)>1):
                    fvec_n[room==2,:,1]=normalize(fvec[room==2,:,1],-1,1,m=m[1][0],M=m[1][1])
                    fvec_n[room==2,:,2]=normalize(fvec[room==2,:,2],-1,1,m=m[1][2],M=m[1][3])
                    fvec_n[room==2,:,3]=normalize(fvec[room==2,:,3],-1,1,m=m[1][4],M=m[1][5])

            if(len(subdirslog)>2):
                    fvec_n[room==3,:,1]=normalize(fvec[room==3,:,1],-1,1,m=m[2][0],M=m[2][1])
                    fvec_n[room==3,:,2]=normalize(fvec[room==3,:,2],-1,1,m=m[2][2],M=m[2][3])
                    fvec_n[room==3,:,3]=normalize(fvec[room==3,:,3],-1,1,m=m[2][4],M=m[2][5])
            
    '''
    fscale_n=fscale.copy()

    if(nrmalize):
            fscale_n[room==1,:,1]=normalize(fscale[room==1,:,1],-1,1,m=m[0][0],M=m[0][1])
            fscale_n[room==1,:,2]=normalize(fscale[room==1,:,2],-1,1,m=m[0][2],M=m[0][3])
            fvec_n[room==1,:,3]=normalize(fscale[room==1,:,3],-1,1,m=m[0][4],M=m[0][5])

            if(len(subdirslog)>1):
                    fscale_n[room==2,:,1]=normalize(fscale[room==2,:,1],-1,1,m=m[1][0],M=m[1][1])
                    fscale_n[room==2,:,2]=normalize(fscale[room==2,:,2],-1,1,m=m[1][2],M=m[1][3])
                    fscale_n[room==2,:,3]=normalize(fscale[room==2,:,3],-1,1,m=m[1][4],M=m[1][5])

            if(len(subdirslog)>2):
                    fscale_n[room==3,:,1]=normalize(fscale[room==3,:,1],-1,1,m=m[2][0],M=m[2][1])
                    fscale_n[room==3,:,2]=normalize(fscale[room==3,:,2],-1,1,m=m[2][2],M=m[2][3])
                    fscale_n[room==3,:,3]=normalize(fscale[room==3,:,3],-1,1,m=m[2][4],M=m[2][5])
    '''

    #print('Normalized feature vector for the 58th image')
    #print(fvec_n[58,:,:])

    return fvec_n, label, room

'''
rootdir = 'Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\to-be-Rated'
subdirs=['\\RuralHometo-be-rated_08-Jun-2017']
roomdirs=['\\BR','\\Kitchen','\\LR']

[fvec2_n,label2,room2]=featExtract2(rootdir,subdirs,roomdirs)
'''

"\nrootdir = 'Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\to-be-Rated'\nsubdirs=['\\RuralHometo-be-rated_08-Jun-2017']\nroomdirs=['\\BR','\\Kitchen','\\LR']\n\n[fvec2_n,label2,room2]=featExtract2(rootdir,subdirs,roomdirs)\n"

In [ ]:

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\RuralHome_08-Jun-2017\\logs\\fr2_log.txt'
[coor,scle,sze,rot,corners1] = text2feat(logpath)

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\RuralHome_08-Jun-2017\\logs\\fr53_log.txt'
[coor,scle,sze,rot,corners2] = text2feat(logpath)

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\RuralHome_08-Jun-2017\\logs\\fr100_log.txt'
[coor,scle,sze,rot,corners3] = text2feat(logpath)

corners_rural=[corners1,corners2,corners3]


logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\BasicHouse_2017-06-26-to-be-rated\\logs\\fr15_log.txt'
[coor,scle,sze,rot,corners1] = text2feat(logpath)

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\BasicHouse_2017-06-26-to-be-rated\\logs\\fr102_log.txt'
[coor,scle,sze,rot,corners2] = text2feat(logpath)

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\BasicHouse_2017-06-26-to-be-rated\\logs\\fr179_log.txt'
[coor,scle,sze,rot,corners3] = text2feat(logpath)

corners_basic=[corners1,corners2,corners3]


logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\SmallApt_unity_08-Jun-2017\\logs\\fr9_log.txt'
[coor,scle,sze,rot,corners1] = text2feat(logpath)

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\SmallApt_unity_08-Jun-2017\\logs\\fr56_log.txt'
[coor,scle,sze,rot,corners2] = text2feat(logpath)

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\SmallApt_unity_08-Jun-2017\\logs\\fr100_log.txt'
[coor,scle,sze,rot,corners3] = text2feat(logpath)

corners_small=[corners1,corners2,corners3]

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\BriansHouse-to-be-rated_2017-06-23\\logs\\fr39_log.txt'
[coor,scle,sze,rot,corners1] = text2feat(logpath)

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\BriansHouse-to-be-rated_2017-06-23\\logs\\fr110_log.txt'
[coor,scle,sze,rot,corners2] = text2feat(logpath)

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\BriansHouse-to-be-rated_2017-06-23\\logs\\fr215_log.txt'
[coor,scle,sze,rot,corners3] = text2feat(logpath)

corners_brian=[corners1,corners2,corners3]



rootdir = 'Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated'
subdirs=['\\RuralHome_08-Jun-2017']
roomdirs=['\\BR','\\Kitchen','\\LR']
[fvec_rural_tr1,label_rural_tr1,room_rural_tr1]=featExtract2(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_rural)

subdirs=['\\RuralHome_2017-06-16']
roomdirs=['\\BR','\\Kitchen','\\LR']
[fvec_rural_tr2,label_rural_tr2,room_rural_tr2]=featExtract2(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_rural)

subdirs=['\\SmallApt_unity_2017-06-16']
roomdirs=['\\BR','\\Kitchen','\\LR']
[fvec_small_tr1,label_small_tr1,room_small_tr1]=featExtract2(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_small)

subdirs=['\\SmallApt_unity_08-Jun-2017']
roomdirs=['\\BR','\\Kitchen','\\LR']
[fvec_small_tr2,label_small_tr2,room_small_tr2]=featExtract2(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_small)

subdirs=['\\BasicHouse_2017-06-26-to-be-rated']
roomdirs=['\\BR','\\Kitchen','\\LR']
[fvec_basic_tr,label_basic_tr,room_basic_tr]=featExtract2(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_basic)

subdirs=['\\BriansHouse-to-be-rated_2017-06-23']
roomdirs=['\\BR','\\Kitchen','\\LR']
[fvec_brian_tr,label_brian_tr,room_brian_tr]=featExtract2(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_brian)

fvec_rural_tr=np.concatenate((fvec_rural_tr1,fvec_rural_tr2),axis=0)
room_rural_tr=np.concatenate((room_rural_tr1,room_rural_tr2),axis=0)
label_rural_tr=np.concatenate((label_rural_tr1,label_rural_tr2),axis=0)


fvec_small_tr=np.concatenate((fvec_small_tr1,fvec_small_tr2),axis=0)
room_small_tr=np.concatenate((room_small_tr1,room_small_tr2),axis=0)
label_small_tr=np.concatenate((label_small_tr1,label_small_tr2),axis=0)


fvec_tr=np.concatenate((fvec_rural_tr,fvec_small_tr,fvec_basic_tr,fvec_brian_tr),axis=0)
room_tr=np.concatenate((room_rural_tr,room_small_tr,room_basic_tr,room_brian_tr),axis=0)
label_tr=np.concatenate((label_rural_tr,label_small_tr,label_basic_tr,label_brian_tr),axis=0)

plt.hist(label_tr*2,range(2,22,2))
plt.title('Overall')
plt.figure()


In [ ]:
plt.hist(fvol_tr1)
plt.figure()
plt.hist(label_tr1)

In [ ]:
def feat_transform2(fvec,fsize):
    color_levels=np.array([0,.3,.6,.7,.8,.9,.95,1])
    numScenes = fvec.shape[0]
    fvec_stat=np.zeros((numScenes,14))
    fvec_c = np.zeros(14)
    for k in range(numScenes):
        coors=fvec[k,:,:]
        coors=coors[coors[:,0]==1,:]
        coors=coors[:,1:4]

        x=coors[:,0]
        y=coors[:,1]
        z=coors[:,2]

        x=(11+np.round(256*(x+1))).astype(np.int)
        z=(11+np.round(256*(z+1))).astype(np.int)
        y=(y+1)/2.

        top_height=np.zeros(6)
        if(len(y)>0):
            top_height[0]=y[0]
            top_height[1]=np.mean(y[:10])
            top_height[2]=np.mean(y[:20])
            top_height[3]=np.mean(y[:50])
            top_height[4]=np.mean(y[:100])
            top_height[5]=np.mean(y[:1000])


        clutter_img=heatmap(x,y,z)
        clutter_levels=np.histogram(clutter_img,color_levels)[0]

        numObj=len(y)

        fvec_c[0]=numObj/1500
        fvec_c[1:8]=clutter_levels/(555*555/10)
        fvec_c[8:]=top_height

        fvec_stat[k,:]=fvec_c

    fvec_stat_01=fvec_stat/(np.ones((fvec_stat.shape[0],1))*np.max(fvec_stat,axis=0))
    return fvec_stat

In [ ]:


[fvec_tr1,fvec_vlabel_tr1,room_tr1]=featExtract2(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_small)

In [ ]:
from sklearn import tree
#from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.svm import SVC

#regr = linear_model.LinearRegression()
#regr=SVC
regr=OneVsOneClassifier(SVC(C=3.0,random_state=0,kernel='rbf',class_weight='balanced'))

'''
logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\RuralHome_08-Jun-2017\\logs\\fr2_log.txt'
[coor,scle,sze,rot,corners1] = text2feat(logpath)

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\RuralHome_08-Jun-2017\\logs\\fr53_log.txt'
[coor,scle,sze,rot,corners2] = text2feat(logpath)

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\RuralHome_08-Jun-2017\\logs\\fr100_log.txt'
[coor,scle,sze,rot,corners3] = text2feat(logpath)

corners_rural=[corners1,corners2,corners3]
'''


logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\SmallApt_unity_08-Jun-2017\\logs\\fr9_log.txt'
[coor,scle,sze,rot,corners1] = text2feat(logpath)
print(corners1)

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\SmallApt_unity_08-Jun-2017\\logs\\fr56_log.txt'
[coor,scle,sze,rot,corners2] = text2feat(logpath)

logpath='Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated\\SmallApt_unity_08-Jun-2017\\logs\\fr100_log.txt'
[coor,scle,sze,rot,corners3] = text2feat(logpath)

corners_small=[corners1,corners2,corners3]

rootdir = 'Z:\\Grants\\Image Project\\Data_Sets\\Synthetic\\Rated'
subdirs=['\\SmallApt_unity_2017-06-16']
roomdirs=['\\BR','\\Kitchen','\\LR']

[fvec_tr1,label_tr1,room_tr1]=featExtract2(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_small)
fvec_tr1=feat_transform(fvec_tr1)
[y_cv, y_pred]=cross_val(regr,fvec_tr1,label_tr1) 


subdirs=['\\SmallApt_unity_08-Jun-2017']
[fvec_tr2,label_tr2,room_tr2]=featExtract2(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_small)
fvec_tr2=feat_transform(fvec_tr2)
[y_cv, y_pred]=cross_val(regr,fvec_tr2,label_tr2) 


fvec_tr=np.concatenate((fvec_tr1,fvec_tr2),axis=0)
room_tr=np.concatenate((room_tr1,room_tr2),axis=0)
label_tr=np.concatenate((label_tr1,label_tr2),axis=0)

[y_cv, y_pred]=cross_val(regr,fvec_tr,label_tr) 
print('CV results for new Rural Home images')
print(np.mean((np.abs(np.round(y_pred)-y_cv)<=1).astype(np.float)))
print(cross_val2(regr,fvec_tr,label_tr,cv=5) )

plt.hist(label_tr[room_tr==1]*2,range(2,22,2))
plt.figure()
plt.hist(label_tr[room_tr==2]*2,range(2,22,2))
plt.figure()
plt.hist(label_tr[room_tr==3]*2,range(2,22,2))


In [ ]:
print(fvec_tr.shape)
[y_cv, y_pred]=cross_val(regr,fvec_tr[room_tr==1],label_tr[room_tr==1]) 
print('CV results for new Rural Home images')
print(np.mean((np.abs(np.round(y_pred)-y_cv)<=1).astype(np.float)))
[y_cv, y_pred]=cross_val(regr,fvec_tr[room_tr==2],label_tr[room_tr==2]) 
print('CV results for new Rural Home images')
print(np.mean((np.abs(np.round(y_pred)-y_cv)<=1).astype(np.float)))
[y_cv, y_pred]=cross_val(regr,fvec_tr[room_tr==3],label_tr[room_tr==3]) 
print('CV results for new Rural Home images')
print(np.mean((np.abs(np.round(y_pred)-y_cv)<=1).astype(np.float)))
[y_cv, y_pred]=cross_val(regr,fvec_tr,label_tr) 
print('CV results for new Rural Home images')
print(np.mean((np.abs(np.round(y_pred)-y_cv)<=1).astype(np.float)))
print(cross_val2(regr,fvec_tr,label_tr,cv=5) )

In [ ]:
fvec_rural=np.concatenate((fvec_rural1,fvec_rural2),axis=0)
room_rural=np.concatenate((room_rural1,room_rural2),axis=0)
label_rural=np.concatenate((label_rural1,label_rural2),axis=0)*2
print(fvec_rural.shape)
print(label_rural.shape)

fvec_small=np.concatenate((fvec_small1,fvec_small2),axis=0)
room_small=np.concatenate((room_small1,room_small2),axis=0)
label_small=np.concatenate((label_small1,label_small2),axis=0)*2
print(fvec_small.shape)
print(label_small.shape)

[y_cv, y_pred]=cross_val(regr,fvec_rural,label_rural) 
print('CV results for new Rural Home images')
print(np.mean((np.abs(np.round(y_pred)-y_cv)<=2).astype(np.float)))
print(cross_val2(regr,fvec_rural,label_rural,cv=5) )

[y_cv, y_pred]=cross_val(regr,fvec_small,label_small) 
print('CV results for Small Apt images')
print(np.mean((np.abs(np.round(y_pred)-y_cv)<=2).astype(np.float)))
print(cross_val2(regr,fvec_small,label_small,cv=5) )

plt.hist(label_rural,bins=range(2,20,2))
plt.title('Rural House')
plt.figure()
plt.hist(label_small,bins=range(2,20,2))
plt.title('Small Apt')

In [ ]:
c=np.power(2.,np.arange(2,15))
#g=np.power(2.,np.arange(-3,5))
g=[1]
c=[512]
print(c)

#fvec_test=fvec_rural
#label_test=label_rural

fvec_test=fvec_small
label_test=label_small
cir_test=np.zeros((len(g),len(c)))
for k in range(len(c)):
    for l in range(len(g)):
        regr=OneVsOneClassifier(SVC(C=c[k],random_state=0,kernel='linear',gamma=g[l],class_weight='balanced'))
        [y_cv, y_pred]=cross_val(regr,fvec_test,label_test,cv=2) 
        plt.hist(y_pred,bins=range(2,20,2))
        cir1=np.mean((np.abs(np.round(y_pred/2)-np.round(y_cv/2))<=1).astype(np.float))
        cir_test[l,k]=cir1
    
print(c)
print(cir_test)


In [ ]:

from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

print(label1)
regr=OneVsRestClassifier(SVC(random_state=0),C=3)
regr.fit(fvec1_stat,label1*2)
label2_pred=regr.predict(fvec2_stat)/2
print('Predictions for Rural Home')
print(label2_pred)
err=np.round(label2_pred)-label2
plt.hist(err)
plt.title('Histogram of error')
cir1=np.mean((np.abs(np.round(label2_pred)-label2)<=1).astype(np.float))
cir2=np.mean((np.abs(np.round(label2_pred)-label2)<=2).astype(np.float))
print('CIR-1: %.2f, CIR-2: %.3f'%(cir1,cir2))



In [ ]:


fvec=fvec_tr#np.concatenate((fvec1_stat,fvec2_stat,fvec3_stat),axis=0)
label=label_tr#np.concatenate((label1,label2,label3),axis=0)*2
print(fvec.shape)
print(label.shape)

c=np.power(2.,np.arange(2,10))
g=np.power(2.,np.arange(-3,5))

#c=[np.power(2,8)]
#g=[100,1]
cir1=np.zeros((len(g),len(c)))
for k in range(len(c)):
    for l in range(len(g)):
        regr=OneVsRestClassifier(SVC(C=c[k],gamma=g[l],kernel='rbf',random_state=0,class_weight='balanced'))
        [y_cv, y_pred]=cross_val(regr,fvec,label*2,cv=2) 
        cir1[l,k]=np.mean((np.abs(np.round(y_pred)-y_cv)<=2).astype(np.float))
        
print(cir1)
idx=np.unravel_index(cir1.argmax(), cir1.shape)
print('MAX CIR-1='+str(cir1[idx]),' achieved at C='+str(c[idx[1]])+', gamma='+str(g[idx[0]]))



cir2=np.mean((np.abs(np.round(y_pred)-y_cv)<=2).astype(np.float))
print('CIR-1: %.2f, CIR-2: %.3f'%(cir1,cir2))
print(cross_val2(regr,fvec,label,cv=5))

idx=np.argsort(y_cv)

y_cv=y_cv[idx]
y_pred=y_pred[idx]

plt.plot(range(len(y_cv)),y_cv/2)
plt.plot(range(len(y_cv)),y_pred/2)


Let us estimate CIR values for new images

In [ ]:
def featExtract3(rootdir,subdirs,roomdirs,nrmalize=True,m=[[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]],wrong_feat=False):
    room=np.zeros((50000,1))
    fvec = np.zeros((50000,15000,4))
    frame=np.arange(0,50000)

    for k in range(len(subdirs)):
            for l in range(len(roomdirs)):
                fnames=os.listdir("".join((rootdir,subdirs[k],roomdirs[l])))
                for fname in fnames:
                    if fname.endswith('.txt'):
                        imNumber=int(fname.split('_')[0][2:])+k*1000
                        logpath="".join((rootdir,subdirs[k],roomdirs[l],'\\',fname))
                        if(wrong_feat):
                            [coor,scle,sze,rot,corners] = text2feat2(logpath);
                        else:
                            [coor,scle,sze,rot,corners] = text2feat(logpath);
                        [coor,scle,sze,rot] = prune_data2(coor,scle,sze,rot,corners)
                        fvec[imNumber,:coor.shape[0],0]=1
                        fvec[imNumber,:coor.shape[0],1:4]=coor
                        del coor
                        room[imNumber]=l+1;

    fvec=np.delete(fvec,np.where(room==0)[0],axis=0)
    frame=np.delete(frame,np.where(room==0)[0])
    room=np.delete(room,np.where(room==0)[0])

    print(''.join(('Number of frames is ',str(len(room)))))

    print('Frame numbers')
    print(frame)
    
    print('Rooms of scenes')
    print(room)

    print('Shape of the feature vector')
    print(fvec.shape)

    #print('Feature vector for the 58th image')
    #print(fvec[58,:,:])

    fvec_n=fvec.copy()

    if(nrmalize):
            fvec_n[room==1,:,1]=normalize(fvec[room==1,:,1],-1,1,m=m[0][0],M=m[0][1])
            fvec_n[room==1,:,2]=normalize(fvec[room==1,:,2],-1,1,m=m[0][2],M=m[0][3])
            fvec_n[room==1,:,3]=normalize(fvec[room==1,:,3],-1,1,m=m[0][4],M=m[0][5])

            if(len(subdirslog)>1):
                    fvec_n[room==2,:,1]=normalize(fvec[room==2,:,1],-1,1,m=m[1][0],M=m[1][1])
                    fvec_n[room==2,:,2]=normalize(fvec[room==2,:,2],-1,1,m=m[1][2],M=m[1][3])
                    fvec_n[room==2,:,3]=normalize(fvec[room==2,:,3],-1,1,m=m[1][4],M=m[1][5])

            if(len(subdirslog)>2):
                    fvec_n[room==3,:,1]=normalize(fvec[room==3,:,1],-1,1,m=m[2][0],M=m[2][1])
                    fvec_n[room==3,:,2]=normalize(fvec[room==3,:,2],-1,1,m=m[2][2],M=m[2][3])
                    fvec_n[room==3,:,3]=normalize(fvec[room==3,:,3],-1,1,m=m[2][4],M=m[2][5])
    return fvec_n, room, frame

#print('Normalized feature vector for the 58th image')
#print(fvec_n[58,:,:])

#rootdir='G:\\HoardingImages'
#subdirs=['\\RuralHome']
#roomdirs=['\\BR','\\Kitchen','\\LR']
#[fvec_new_rural,room_new_rural,frame_rural]=featExtract3(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_rural)
#fvec_new_stat_rural=feat_transform(fvec_new_rural)

In [ ]:
'''
logpath='G:\\HoardingImages\\BasicHouse_2017-06-26\\BR\\fr1074_log.txt'
[coor,scle,sze,rot,corners1] = text2feat(logpath)

logpath='G:\\HoardingImages\\BasicHouse_2017-06-26\\Kitchen\\fr100_log.txt'
[coor,scle,sze,rot,corners2] = text2feat(logpath)

logpath='G:\\HoardingImages\\BasicHouse_2017-06-26\\LR\\fr180_log.txt'
[coor,scle,sze,rot,corners3] = text2feat(logpath)

corners_basic=[corners1,corners2,corners3]
'''

rootdir='G:\\HoardingImages'
subdirs=['\\_completed\\RuralHome_2017-06-30']
roomdirs=['\\BR','\\Kitchen','\\LR']
[fvec_rural_test,room_rural_test,frame_rural_test]=featExtract3(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_small)
fvec_rural_test=feat_transform(fvec_rural_test)
print(fvec_rural_test.shape)

In [ ]:
logpath='G:\\HoardingImages\\BriansHouse_2017-06-19\\BR\\fr1935_log.txt'
[coor,scle,sze,rot,corners1] = text2feat(logpath)

logpath='G:\\HoardingImages\\BriansHouse_2017-06-19\\Kitchen\\fr125_log.txt'
[coor,scle,sze,rot,corners2] = text2feat(logpath)

logpath='G:\\HoardingImages\\BriansHouse_2017-06-19\\LR\\fr260_log.txt'
[coor,scle,sze,rot,corners3] = text2feat(logpath)

corners_brian=[corners1,corners2,corners3]

rootdir='G:\\HoardingImages'
subdirs=['\\BriansHouse_2017-06-23']
roomdirs=['\\BR','\\Kitchen','\\LR']
[fvec_new,room_new,frame_new]=featExtract3(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_brian)
fvec_new=feat_transform(fvec_new)
print(fvec_new.shape)

In [ ]:
rootdir='G:\\HoardingImages'
subdirs=['\\RuralHome']
roomdirs=['\\BR','\\Kitchen','\\LR']
[fvec_new_rural,room_new1,frame_rural1]=featExtract3(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_rural,wrong_feat=True)
fvec_new1=feat_transform(fvec_new_rural)

subdirs=['\\RuralHome_lo_CIRS2']
[fvec_new_rural,room_new2,frame_rural2]=featExtract3(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_rural)
fvec_new2=feat_transform(fvec_new_rural)

subdirs=['\\RuralHome_low_CIR3']
[fvec_new_rural,room_new3,frame_rural3]=featExtract3(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_rural)
fvec_new3=feat_transform(fvec_new_rural)

subdirs=['\\RuralHome_low_CIRs']
[fvec_new_rural,room_new4,frame_rural4]=featExtract3(rootdir,subdirs,roomdirs,nrmalize=True,m=corners_rural)
fvec_new4=feat_transform(fvec_new_rural)


In [ ]:
#Classify the images

from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.svm import SVC
regr=OneVsRestClassifier(SVC(C=32,gamma=1,kernel='rbf',random_state=0,class_weight='balanced'))
regr.fit(fvec_tr,label_tr*2)
label_pred=regr.predict(fvec_test)

label_pred2=np.zeros(label_pred.shape)
for rm in range(1,4):
    regr.fit(fvec_tr[room_tr==rm,:],label_tr[room_tr==rm]*2)
    label_pred2[room_test==rm]=regr.predict(fvec_test[room_test==rm,:])
    
labels=np.ceil(label_pred/2)
plt.hist(labels,range(1,11))
plt.title('Cumulative')
plt.figure()
plt.hist(label_pred2,range(0,22,2))
plt.title('Room Specific')
plt.figure()
plt.hist(label_pred2-label_pred)
plt.title('Difference')
plt.figure()
plt.plot(frame_test[:150],label_pred[:150])

print(np.unique(labels))
print(np.histogram(labels,range(1,11)))

In [ ]:
frames=[frame_test[room_test==1],frame_test[room_test==2],frame_test[room_test==3]]
print(label_pred[4895])

In [ ]:
import shutil
rootdir='G:\\HoardingImages\\SmallApt_2017-06-29'
subdirs=['\\BR','\\Kitchen','\\LR']
im_num=6
frames=[frame_test[room_test==1],frame_test[room_test==2],frame_test[room_test==3]]
labels=np.ceil(label_pred/2).astype(np.int)
labels=[labels[room_test==1],labels[room_test==2],labels[room_test==3]]
destdir=rootdir+'-rated'
if not os.path.exists(destdir+'\\logs'):
        os.makedirs(destdir+'\\logs')
        
for k in range(1,len(subdirs)):
    if not os.path.exists(destdir+subdirs[k]):
        for cir in range(1,10):
            os.makedirs(destdir+subdirs[k]+'\\'+str(cir))
    for fr in range(len(frames[k])):
        fr_num=frames[k][fr]
        fr_label=labels[k][fr]
        if(fr!=0):
            for l in range(im_num):
                targetdir="".join((rootdir,subdirs[k],'\\cam',str(l+1),'_fr',str(fr_num),'.png'))
                destdirr="".join((destdir,subdirs[k],'\\',str(fr_label),'\\cam',str(l+1),'_fr',str(fr_num),'.png'))
                shutil.copy2(targetdir,destdirr)
            logTarget="".join((rootdir,subdirs[k],'\\fr',str(fr_num),'_log.txt'))
            logDest="".join((destdir,'\\logs\\fr',str(fr_num),'_log.txt'))
            shutil.copy2(logTarget,logDest)


In [ ]:
fvec_tr=np.concatenate((fvec_rural,fvec_small),axis=0)
room_tr=np.concatenate((room_rural,room_small),axis=0)
label_tr=np.concatenate((label_rural,label_small),axis=0)*2
print(fvec_tr.shape)
print(label_tr.shape)

In [ ]:
def selectImages(regr,fvec_tr,label_tr,fvec_test,room_test,im_count,min_cir=1,max_cir=9):
    regr.fit(fvec_tr,label_tr)
    label_new=regr.predict(fvec_test)
    rooms=np.unique(room_test)
    idx=-1*np.ones((len(rooms),im_count*(max_cir-min_cir+1)))
    
    for rm in rooms:
        cur_pos=0
        lbl=label_new.copy()
        lbl[room_test!=rm]=-1
        for k in range(min_cir,max_cir+1):
            idx_c=np.where(np.logical_and(lbl>=2*k,lbl<2*k+2).astype(np.float))[0]
            len_c=len(idx_c)
            if(len_c<im_count*k-cur_pos):
                idx[rm-1,cur_pos:cur_pos+len_c]=idx_c
                cur_pos=cur_pos+len_c
            else:
                print(np.floor(np.linspace(0,len_c-1,im_count*k-cur_pos)).astype(np.int))
                idx[rm-1,cur_pos:im_count*k]=idx_c[np.floor(np.linspace(0,len_c-1,im_count*k-cur_pos)).astype(np.int)]
                cur_pos=im_count*k
    return idx.astype(np.int),label_new

regr=OneVsRestClassifier(SVC(C=512,kernel='linear',random_state=0,class_weight='balanced'))
[idx, label_new_basic]=selectImages(regr,fvec_rural,label_rural,fvec_new_basic,room_new_basic,im_count=12)
print('printing labels')
print(np.histogram(label_new_basic,bins=range(2,20,2)))
print(idx)

Image Merger

In [ ]:
import datetime
import os
import skimage.io
import shutil
def saveImages(rootdir,subdirs,fr_num,im_num=6):
    destdir=rootdir+'-to-be-rated'
    if not os.path.exists(destdir+'\\logs'):
            os.makedirs(destdir+'\\logs')
    for k in range(len(subdirs)):
        if not os.path.exists(destdir+subdirs[k]):
            for cir in range(1,10):
                os.makedirs(destdir+subdirs[k]+'\\'+str(cir))
            os.makedirs(destdir+subdirs[k]+'\\type1')
        for fr in fr_num[k,:]:
            if(fr!=-1):
                if(im_num==8):
                    im_subs=[None]*8
                    for l in range(8):
                        im_subs[l]=skimage.io.imread("".join((rootdir,subdirs[k],'\\cam',str(l+1),'_fr',str(fr),'.png')))
                    im=np.zeros((1034,2600,3))
                    im[:512,522:1034,:]=im_subs[4];
                    im[:512,1044:1556,:]=im_subs[3];
                    im[:512,1566:2078,:]=im_subs[5];
                    im[522:1034,:512,:]=im_subs[1];
                    im[522:1034,522:1034,:]=im_subs[6];
                    im[522:1034,1044:1556,:]=im_subs[0];
                    im[522:1034,1566:2078,:]=im_subs[7];
                    im[522:1034,2088:,:]=im_subs[2];
                else:
                    im_subs=[None]*6
                    for l in range(6):
                        im_subs[l]=skimage.io.imread("".join((rootdir,subdirs[k],'\\cam',str(l+1),'_fr',str(fr),'.png')))
                    im=np.zeros((1034,1556,3))
                    im[:512,:512,:]=im_subs[4];
                    im[:512,522:1034,:]=im_subs[3];
                    im[:512,1044:,:]=im_subs[5];
                    im[522:1034,:512,:]=im_subs[1];
                    im[522:1034,522:1034,:]=im_subs[0];
                    im[522:1034,1044:,:]=im_subs[2];
                skimage.io.imsave("".join((destdir,subdirs[k],'\\type1\\fr',str(fr),'.png')),im.astype(np.uint8))
                logTarget="".join((rootdir,subdirs[k],'\\fr',str(fr),'_log.txt'))
                logDest="".join((destdir,'\\logs\\fr',str(fr),'_log.txt'))
                shutil.copy2(logTarget,logDest)

rootdir='G:\\HoardingImages\\BasicHouse_2017-06-26'
subdirs=['\\BR','\\Kitchen','\\LR']
fr_num=frame_new_basic[idx]
fr_num[idx==-1]=-1
print(fr_num)
saveImages(rootdir,subdirs,fr_num)


